# Possible Approach

## Hypothesis, does adding Pos, Neg, and Neu values from Sentiment Analysis improve the original model??

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../TextFiles/moviereviews.tsv', sep='\t')
df.head()

label                                             review
0   neg  how do films like mouse hunt get into theatres...
1   neg  some talented actresses are blessed with a dem...
2   pos  this has been an extraordinary year for austra...
3   pos  according to hollywood movies made in last few...
4   neg  my first press screening of 1998 and already i...

In [2]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
df.dropna(inplace=True)

blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df.drop(blanks, inplace=True)

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [4]:
df.head()

label                                             review
0   neg  how do films like mouse hunt get into theatres...
1   neg  some talented actresses are blessed with a dem...
2   pos  this has been an extraordinary year for austra...
3   pos  according to hollywood movies made in last few...
4   neg  my first press screening of 1998 and already i...

In [5]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))

In [6]:
df.head()

label                                             review  \
0   neg  how do films like mouse hunt get into theatres...   
1   neg  some talented actresses are blessed with a dem...   
2   pos  this has been an extraordinary year for austra...   
3   pos  according to hollywood movies made in last few...   
4   neg  my first press screening of 1998 and already i...   

                                              scores  
0  {'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...  
1  {'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...  
2  {'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...  
3  {'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...  
4  {'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...

In [7]:
df['positive'] = df['scores'].apply(lambda score_dict: score_dict['pos'])
df['negative'] = df['scores'].apply(lambda score_dict: score_dict['neg'])
df['neutral'] = df['scores'].apply(lambda score_dict: score_dict['neu'])
df['compound']  =df['scores'].apply(lambda score_dict: score_dict['compound'])

In [13]:
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

In [14]:
df.head()

label                                             review  \
0   neg  how do films like mouse hunt get into theatres...   
1   neg  some talented actresses are blessed with a dem...   
2   pos  this has been an extraordinary year for austra...   
3   pos  according to hollywood movies made in last few...   
4   neg  my first press screening of 1998 and already i...   

                                              scores  positive  negative  \
0  {'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...     0.101     0.121   
1  {'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...     0.105     0.120   
2  {'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...     0.150     0.068   
3  {'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...     0.147     0.071   
4  {'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...     0.093     0.091   

   neutral  compound comp_score  
0    0.778   -0.9125        neg  
1    0.775   -0.8618        neg  
2    0.781    0.9951        pos  
3    0.782    0.9972        pos  
4    0.817   -0.2484        neg

In [15]:
print(df.iloc[15]['review'])

here's a rarity : a children's film that attempts to tackle a weighty subject , is there a god ? 
done well , it could have been a gem among the wasteland of modern children's cinema . 
unfortunately , it isn't . 
with jumbled messages , and an unclear audience , wide awake was better left asleep . 
fifth grader joshua beal ( joseph cross ) is in the middle of a moral crisis . 
his beloved grandfather ( robert loggia ) has died , and joshua has begun a quest . 
he wants to find god , to discover why bad things happen . 
this religious quest is slightly disturbing for his parents ( dana delany and denis leary ) , but they do their best to cope with their son as he explores different religious faiths . 
at his catholic school , his favorite teacher , sister terry ( rosie o'donnell ) , tries to give him guidance , but this is a journey he must make on his own . 
meanwhile , he is having the most momentous year of his life . 
he has several adventures with his daredevil best friend dave ( 

In [16]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [17]:
accuracy_score(df['label'],df['comp_score'])

0.6357069143446853

In [18]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938

